## 协同过滤：

基于用户的协同过滤算法是通过用户的历史行为数据发现用户对商品或内容的喜欢(如商品购买，收藏，内容评论或分享)，并对这些喜好进行度量和打分。根据不同用户对相同商品或内容的态度和偏好程度**计算用户之间的关系**。**在有相同喜好的用户间进行商品推荐**。简单的说就是如果A,B两个用户都购买了x,y,z三本图书，并且给出了5星的好评。那么A和B就属于同一类用户。可以将A看过的图书w也推荐给用户B。

https://www.jianshu.com/p/d15ba37755d1


<img src="image_github/collaborative_filter.png" width="500" height="500">

**挑战：新用户，新商品，新社区。以及，大规模的数据，矩阵的稀疏。**

### 1. 协同过滤方法：

1. **Memory-based:** 

    **基于项目的(Item一based)协同过滤是根据用户对相似项目的评分数据预测目标项目的评分。**

    **基于用户的(User-based)协同过滤是通过相似用户对项目的评分推荐给目标用户。**


2. **Model-based:**
    
    **使用机器学习算法进行预测。例如：矩阵分解**


### 2. 矩阵分解 Matrix Factorisation - 隐含语义模型：

矩阵分解可以用于实现隐含语义模型。例如在电影推荐系统中，将用户-电影-等级矩阵分解成用户矩阵以及电影矩阵，其中，每个用户以及电影由一个向量表示 类似于word-embedding（下图所示）。

<img src="image_github/m_f.png" width="500" height="500">

$p_u$: 用户u。

$q_i$: 物品i。

$r_{ui} = q_i^T p_u$: 用户u对物品i的评价等级。

**注意：由于rating矩阵中存在缺失值，因此，不能使用奇异值分解方法进行矩阵分解。首先，需要对缺失值进行填充，然后在利用SVD进行矩阵分解。使用方法：交替最小二乘法。**

### 3. 交替最小二乘法 ALS：

#### 1. 矩阵分解损失函数为：

<img src="image_github/object_function.png" width="500" height="500">

**注意：由于 $p_u$，$q_i$均未知，无法使用最小二乘法，梯度下降等算法。解决办法：交替最小二乘法。**

#### 2. 交替最小二乘法：
固定其中一项然后不断交替直到代价函数小于阈值，例如：首先固定物品向量p，根据最小二乘法求q。然后，根据得到的q求p，以此类推。重复上述步骤，直至收敛。由于上述步骤为交替对p以及q使用**最小二乘法**，因此最后会收敛。

<img src="image_github/fix_p.png" width="300" height="300">

 或者使用**梯度下降**更新参数公式如下所示：

<img src="image_github/m_f_equation.png" width="500" height="500">

**Evaluator: MAE 或 RMSE**

<img src="image_github/evaluator.png" width="500" height="500">

### 4. pyspark：

In [38]:
import os
import subprocess
def module(*args):        
    if isinstance(args[0], list):        
        args = args[0]        
    else:        
        args = list(args)        
    (output, error) = subprocess.Popen(['/usr/bin/modulecmd', 'python'] + args, stdout=subprocess.PIPE).communicate()
    exec(output)    
module('load', 'apps/java/jdk1.8.0_102/binary')    
os.environ['PYSPARK_PYTHON'] = os.environ['HOME'] + '/.conda/envs/jupyter-spark/bin/python'

In [39]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Collaborative Filtering RecSys") \
    .getOrCreate()

sc = spark.sparkContext

In [40]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

**使用movielen数据，其格式如下所示：**
<img src="image_github/movie_format.png" width="400" height="400">

**pyspark.sql.Row：类似于字典的对象，可以通过Row(index = 'value')进行创建，可通过row.index对象形式或row['index']字典形式访问值。**

In [41]:
lines = spark.read.text("../Data/MovieLens100k.data").rdd
lines.take(5)

[Row(value='196\t242\t3\t881250949'),
 Row(value='186\t302\t3\t891717742'),
 Row(value='22\t377\t1\t878887116'),
 Row(value='244\t51\t2\t880606923'),
 Row(value='166\t346\t1\t886397596')]

**注意：spark.read.text()创建dataframe然后转换成rdd，其中，每一行为Row对象。直接使用sc.textFile()创建rdd，每一行为string类型。**

In [42]:
parts = lines.map(lambda row: row.value.split("\t"))

ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

ratings.show(10)

+-------+------+---------+------+
|movieId|rating|timestamp|userId|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
|    474|   4.0|884182806|   298|
|    265|   2.0|881171488|   115|
|    465|   5.0|891628467|   253|
|    451|   3.0|886324817|   305|
|     86|   3.0|883603013|     6|
+-------+------+---------+------+
only showing top 10 rows



In [43]:
ratings.select(ratings.userId).distinct().count()

943

In [44]:
ratings.groupBy('userId').count().count()

943

In [45]:
(training, test) = ratings.randomSplit([0.9, 0.1], 1234) # 1234 random seed

#### coldStartStrategy：测试集可能包含训练集未出现的用户或电影，'drop'参数会对这些数据进行忽略。

In [46]:
als = ALS(maxIter=10, regParam=0.1, rank=3, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",predictionCol="prediction")
mae = evaluator.evaluate(predictions)

print(mae)

0.7322716775792905


#### 给用户推荐电影：

In [47]:
userRecs = model.recommendForAllUsers(10)
userRecs.show(5)

userRecs.count()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[1554, 6.7041345...|
|   463|[[1175, 4.64767],...|
|   833|[[1463, 5.290497]...|
|   496|[[1463, 5.6878576...|
|   148|[[1463, 5.7998896...|
+------+--------------------+
only showing top 5 rows



943

#### 对特定用户进行推荐电影：

In [48]:
users = ratings.select('userId').distinct() # als.getUserCol()
users.show(5)

u1 = users.filter(users.userId.between(20, 25))
userSubsetRecs = model.recommendForUserSubset(u1, 6)

userSubsetRecs.show()

+------+
|userId|
+------+
|    26|
|    29|
|   474|
|   191|
|    65|
+------+
only showing top 5 rows

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    22|[[1463, 5.755835]...|
|    20|[[1554, 5.4050055...|
|    23|[[1463, 5.067517]...|
|    25|[[814, 5.2719564]...|
|    24|[[814, 5.761121],...|
|    21|[[1463, 5.342782]...|
+------+--------------------+



#### 根据电影推荐用户：

In [49]:
movies = ratings.select('movieId').distinct() # 1682.
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show(10)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|    463|[[4, 4.917675], [...|
|   1591|[[519, 5.2030096]...|
|    496|[[688, 6.1449723]...|
|   1238|[[688, 4.5332084]...|
|   1342|[[239, 4.05011], ...|
|    833|[[507, 4.5306754]...|
|    471|[[688, 4.9977026]...|
|   1580|[[688, 2.0568476]...|
|   1088|[[127, 5.0710692]...|
|   1645|[[4, 6.089705], [...|
+-------+--------------------+
only showing top 10 rows



#### ParamGridBuilder 与 CrossValidator：

In [50]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

als = ALS(maxIter=10, rank=10, userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

paramGrid = ParamGridBuilder().addGrid(als.regParam, [0.01, 0.1, 0.3, 0.5]).build()
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

cvModel = crossval.fit(training)
prediction = cvModel.transform(test)
rmse = evaluator.evaluate(predictions)
print(rmse)

0.9189286346635134


In [51]:
params = cvModel.getEstimatorParamMaps()
avgMetrics = cvModel.avgMetrics
print(avgMetrics)
print(cvModel.bestModel)

all_params = list(zip(params, avgMetrics))
best_param = sorted(all_params, key=lambda x: x[1], reverse=True)[0] # good => bad.

print(best_param)

[1.1693286927952753, 0.9479240219498275, 0.9856798579121662, 1.0723463487332474]
ALS_4604aa70b1f0f7cfdbac
({Param(parent='ALS_4604aa70b1f0f7cfdbac', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, 1.1693286927952753)
